# Day 21: Monkey Math

[https://adventofcode.com/2022/day/21](https://adventofcode.com/2022/day/21)

## Description

### Part One

The [monkeys](https://adventofcode.com/2022/day/11) are back! You're worried they're going to try to steal your stuff again, but it seems like they're just holding their ground and making various monkey noises at you.

Eventually, one of the elephants realizes you don't speak monkey and comes over to interpret. As it turns out, they overheard you talking about trying to find the grove; they can show you a shortcut if you answer their _riddle_.

Each monkey is given a _job_: either to _yell a specific number_ or to _yell the result of a math operation_. All of the number-yelling monkeys know their number from the start; however, the math operation monkeys need to wait for two other monkeys to yell a number, and those two other monkeys might _also_ be waiting on other monkeys.

Your job is to _work out the number the monkey named `root` will yell_ before the monkeys figure it out themselves.

For example:

    root: pppw + sjmn
    dbpl: 5
    cczh: sllz + lgvd
    zczc: 2
    ptdq: humn - dvpt
    dvpt: 3
    lfqf: 4
    humn: 5
    ljgn: 2
    sjmn: drzm * dbpl
    sllz: 4
    pppw: cczh / lfqf
    lgvd: ljgn * ptdq
    drzm: hmdt - zczc
    hmdt: 32
    

Each line contains the name of a monkey, a colon, and then the job of that monkey:

*   A lone number means the monkey's job is simply to yell that number.
*   A job like `aaaa + bbbb` means the monkey waits for monkeys `aaaa` and `bbbb` to yell each of their numbers; the monkey then yells the sum of those two numbers.
*   `aaaa - bbbb` means the monkey yells `aaaa`'s number minus `bbbb`'s number.
*   Job `aaaa * bbbb` will yell `aaaa`'s number multiplied by `bbbb`'s number.
*   Job `aaaa / bbbb` will yell `aaaa`'s number divided by `bbbb`'s number.

So, in the above example, monkey `drzm` has to wait for monkeys `hmdt` and `zczc` to yell their numbers. Fortunately, both `hmdt` and `zczc` have jobs that involve simply yelling a single number, so they do this immediately: `32` and `2`. Monkey `drzm` can then yell its number by finding `32` minus `2`: _`30`_.

Then, monkey `sjmn` has one of its numbers (`30`, from monkey `drzm`), and already has its other number, `5`, from `dbpl`. This allows it to yell its own number by finding `30` multiplied by `5`: _`150`_.

This process continues until `root` yells a number: _`152`_.

However, your actual situation involves <span title="Advent of Code 2022: Now With Considerably More Monkeys">considerably more monkeys</span>. _What number will the monkey named `root` yell?_


In [4]:
from pathlib import Path
from typing import Generator

TEST = """root: pppw + sjmn
dbpl: 5
cczh: sllz + lgvd
zczc: 2
ptdq: humn - dvpt
dvpt: 3
lfqf: 4
humn: 5
ljgn: 2
sjmn: drzm * dbpl
sllz: 4
pppw: cczh / lfqf
lgvd: ljgn * ptdq
drzm: hmdt - zczc
hmdt: 32""".splitlines()

EXPECTED_1 = 152
EXPECTED_2 = None

DATA = Path("input/data21.txt").read_text().splitlines()

import operator
from typing import Callable

OPERATORS = {
    "+": operator.add,
    "-": operator.sub,
    "*": operator.mul,
    "/": operator.floordiv,
}


def parse(data: list[str]) -> dict[str, int | tuple[Callable, str, str]]:
    monkeys = {}
    for line in data:
        if not line:
            continue
        monkey, _, value = line.partition(": ")
        if value.isdigit():
            monkeys[monkey] = int(value)
        else:
            ma, op, mb = tuple(value.split())
            monkeys[monkey] = (OPERATORS[op], ma, mb)
    return monkeys


def score_1(data: list[str]) -> int:
    monkeys = parse(data)
    done = {m: v for m, v in monkeys.items() if isinstance(v, int)}
    queue = {m: v for m, v in monkeys.items() if not isinstance(v, int)}

    while "root" not in done and queue:
        for q in list(queue):
            op, a, b = queue[q]
            if a in done and b in done:
                done[q] = op(done[a], done[b])
                del queue[q]

    return done["root"]


test_score = score_1(TEST)
print("test", test_score)
assert test_score == EXPECTED_1
print("part 1", score_1(DATA))

test 152
part 1 56490240862410


<div class="alert alert-info">Pretty straightforward. Build a dict of known values and another one with the required operations then just keep search for and removing operations we can do.
I could have done something fancy with a tree but wasn't needed.</div>


### Part Two

Due to some kind of monkey-elephant-human mistranslation, you seem to have misunderstood a few key details about the riddle.

First, you got the wrong job for the monkey named `root`; specifically, you got the wrong math operation. The correct operation for monkey `root` should be `=`, which means that it still listens for two numbers (from the same two monkeys as before), but now checks that the two numbers _match_.

Second, you got the wrong monkey for the job starting with `humn:`. It isn't a monkey - it's _you_. Actually, you got the job wrong, too: you need to figure out _what number you need to yell_ so that `root`'s equality check passes. (The number that appears after `humn:` in your input is now irrelevant.)

In the above example, the number you need to yell to pass `root`'s equality test is _`301`_. (This causes `root` to get the same number, `150`, from both of its monkeys.)

_What number do you yell to pass `root`'s equality test?_


In [5]:
# q = a + b => a = q - b, b = q - a
# q = a - b => a = q - b, b = a - q
# q = a * b => a = q / b, b = q / a
# q = a /b => a = q * b, b = a / q
REVOPS = {
    operator.add: operator.sub,
    operator.sub: operator.add,
    operator.mul: operator.floordiv,
    operator.floordiv: operator.mul,
}
REVOPS2 = {
    operator.add: operator.sub,
    operator.sub: (lambda q, a: a - q),
    operator.mul: operator.floordiv,
    operator.floordiv: (lambda q, a: a // q),
}


def score_2(data: list[str]) -> int:
    monkeys = parse(data)
    del monkeys["humn"]
    done = {m: v for m, v in monkeys.items() if isinstance(v, int)}
    queue = {m: v for m, v in monkeys.items() if not isinstance(v, int)}
    computed = done.keys()  # Live view of dict keys
    rootop, roota, rootb = queue.pop("root")

    while roota not in done and rootb not in done:
        for q in list(queue):
            op, a, b = queue[q]
            if {a, b} < computed:
                done[q] = op(done[a], done[b])
                del queue[q]

    if roota in done:
        done["root"] = done[rootb] = done[roota]
    else:
        done["root"] = done[roota] = done[rootb]

    while "humn" not in done:
        for q in list(queue):
            op, a, b = queue[q]
            if {a, b} <= computed:
                done[q] = op(done[a], done[b])
                del queue[q]
            elif {q, b} <= computed:
                done[a] = REVOPS[op](done[q], done[b])
                del queue[q]
            elif {q, a} <= computed:
                done[b] = REVOPS2[op](done[q], done[a])
                del queue[q]

    return done["humn"]


EXPECTED_2 = 301

if EXPECTED_2 is not None:
    test_score = score_2(TEST)
    assert test_score == EXPECTED_2
    print("test", test_score)
    print("part 2", score_2(DATA))


test 301
part 2 3403989691757


<div class="alert alert-info">Extra complexity here to not calculate 'root' and instead calculate either of the two values leading to root. Then for each q,a,b combination where we have two out of three calculate the third until we get to a value for 'humn'. Some cuteness using the live nature of the `dict.keys()` view combined with set comparison, but hey, I'm having fun.</div>